<a href="https://colab.research.google.com/github/skarfie123/detectron2_pdm/blob/main/Detectron2_PDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Detectron2

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
# !pip install torch==1.7.1 torchvision==0.8.2
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

!pip install jsonpickle
!pip install resnest --pre

from google.colab import output as colab_output
colab_output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
!pip freeze | grep "torch"

# Import Detectron2 and PDM

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import matplotlib.pyplot as plt
import os, json, cv2, random, math, sys, subprocess, logging
from collections import OrderedDict
from importlib import reload
from google.colab.patches import cv2_imshow
from pprint import pprint

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.data import build, build_detection_test_loader, MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, DatasetEvaluator, DatasetEvaluators
from detectron2.utils.visualizer import Visualizer

In [ ]:
# tf.test.gpu_device_name()
!nvidia-smi

In [ ]:
!git clone https://github.com/skarfie123/detectron2_pdm

In [ ]:
import detectron2_pdm
from detectron2_pdm import PDM_Evaluator as pdm
PDM_Evaluator = pdm.PDM_Evaluator
from detectron2_pdm import CustomTrainer as ct
CustomTrainer = ct.CustomTrainer
from detectron2_pdm import CustomConfig as cc
CustomConfig = cc.CustomConfig
from detectron2_pdm import Datasets
from detectron2_pdm import Console
from detectron2_pdm import Main
from detectron2_pdm import Visualise
Datasets.clear()

In [ ]:
from google.colab import output as colab_output
def beep():
    colab_output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
beep()

# Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir outputs
for dir in next(os.walk("/content/gdrive/My Drive/4YP Output/detectron/"))[1]:
    os.mkdir("/content/outputs/"+dir)

In [ ]:
# !rm -rf outputs
# !rm -rf /content/outputs/ground0/*
folder = "M_train"
os.system(f"cp -rf /content/gdrive/My\ Drive/4YP\ Output/detectron/{folder}/* /content/outputs/{folder}/")
beep()

In [ ]:
folder = "M_train"
os.system(f"cp -rf /content/outputs/{folder} /content/gdrive/My\ Drive/4YP\ Output/detectron")
beep()

In [ ]:
from tqdm import tqdm
pbar = tqdm(["M_train", "M_test", "G_test", "V_test"])
for folder in pbar:
    pbar.set_description(folder)
    os.system(f"cp -rf /content/gdrive/My\ Drive/4YP\ Output/detectron/{folder}/* /content/outputs/{folder}/")
beep()

In [ ]:
from tqdm import tqdm
pbar = tqdm(["M_test", "G_test", "V_test"])
for folder in pbar:
    pbar.set_description(folder)
    os.system(f"cp -rf /content/outputs/{folder} /content/gdrive/My\ Drive/4YP\ Output/detectron")
beep()

# Configs

In [ ]:
# # !wget https://raw.githubusercontent.com/zhanghang1989/ResNeSt/master/d2/configs/COCO-InstanceSegmentation/mask_cascade_rcnn_ResNeSt_200_FPN_dcn_syncBN_all_tricks_3x.yaml -O resnest.yaml
# # # !wget https://s3.us-west-1.wasabisys.com/resnest/detectron/mask_cascade_rcnn_ResNeSt_200_FPN_dcn_syncBN_all_tricks_3x-e1901134.pth -O resnest.
# !wget https://dl.fbaipublicfiles.com/detectron2/Misc/cascade_mask_rcnn_R_50_FPN_3x/144998488/model_final_480dd8.pkl -O cascade.pth

In [ ]:
from collections import namedtuple
Config = namedtuple("Config", ["trainingConfig", "testingConfigs", "saveInterval", "model", "modelWeights"])
MRCNN = Config(
    trainingConfig=cc.ConfigSet(
        category=cc.Category.MERGED,
        imageset=cc.IMAGESET_ORIGINAL_IMAGES,
        dataset="M",
        numClasses=11,
        pdmClasses=[1,2,3,4,6,7,8,9,10],
        folder="M_train",
    ),
    testingConfigs=[
        cc.ConfigSet(
            category=cc.Category.MERGED,
            imageset=cc.IMAGESET_ORIGINAL_IMAGES,
            dataset="M",
            numClasses=11,
            pdmClasses=[1,2,3,4,6,7,8,9,10],
            folder="M_test",
        ),
        cc.ConfigSet(
            category=cc.Category.GROUND,
            imageset=cc.IMAGESET_MASK_GROUND,
            dataset="G",
            numClasses=7,
            pdmClasses=[6,7,8,9,10],
            folder="G_test",
        ),
        cc.ConfigSet(
            category=cc.Category.VERTICAL,
            imageset=cc.IMAGESET_MASK_VERTICAL,
            dataset="V",
            numClasses=6,
            pdmClasses=[1,2,3,4],
            folder="V_test",
        ),
    ],
    saveInterval=1000,
    model = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml",
    modelWeights = ""
)
CMRCNN = Config(
    trainingConfig=cc.ConfigSet(
        category=cc.Category.MERGED,
        imageset=cc.IMAGESET_ORIGINAL_IMAGES,
        dataset="M",
        numClasses=11,
        pdmClasses=[1,2,3,4,6,7,8,9,10],
        folder="M-CMRCNN_train",
    ),
    testingConfigs=[
        cc.ConfigSet(
            category=cc.Category.MERGED,
            imageset=cc.IMAGESET_ORIGINAL_IMAGES,
            dataset="M",
            numClasses=11,
            pdmClasses=[1,2,3,4,6,7,8,9,10],
            folder="M-CMRCNN_test",
        ),
        cc.ConfigSet(
            category=cc.Category.GROUND,
            imageset=cc.IMAGESET_MASK_GROUND,
            dataset="G",
            numClasses=7,
            pdmClasses=[6,7,8,9,10],
            folder="G-CMRCNN_test",
        ),
        cc.ConfigSet(
            category=cc.Category.VERTICAL,
            imageset=cc.IMAGESET_MASK_VERTICAL,
            dataset="V",
            numClasses=6,
            pdmClasses=[1,2,3,4],
            folder="V-CMRCNN_test",
        ),
    ],
    saveInterval=1000,
    model = "Misc/cascade_mask_rcnn_R_50_FPN_3x.yaml",
    modelWeights = ""
)
        # model="/content/mask_cascade_rcnn_ResNeSt_200_FPN_dcn_syncBN_all_tricks_3x.yaml",
        # modelWeights=""
        # modelWeights="/content/resnest200.pth"

In [ ]:
def setupCC():
    config = MRCNN
    CustomConfig.set(
        trainingConfig=config.trainingConfig,
        testingConfigs=config.testingConfigs,
        driveOutputs="/content/gdrive/My\\ Drive/4YP\\ Output/detectron",
        driveDatasets="/content/gdrive/MyDrive/Share/4YPDatasets",
        saveInterval=config.saveInterval,
        model = config.model,
        modelWeights = config.modelWeights
    )
    Datasets.register(CustomConfig.trainingConfig.imageset, CustomConfig.trainingConfig.dataset)
    for cs in CustomConfig.testingConfigs:
        Datasets.register(cs.imageset, cs.dataset)
    beep()
setupCC()

In [ ]:
# subprocess.call(
#     f"cp {CustomConfig.driveDatasets}/resnest.zip /content/",
#     shell=True,
# )
# subprocess.call(f"unzip resnest.zip > /dev/null", shell=True)
# subprocess.call(
#     f"cp {CustomConfig.driveDatasets}/resnest200.pth /content/",
#     shell=True,
# )

In [ ]:
# !pip install gdown
# !gdown https://drive.google.com/uc?id=1uSSvy4V7ALjousc7Tqy3tlGH7nLUijhG -O resnest200.pth

In [ ]:
for tc in CustomConfig.testingConfigs:
    DatasetCatalog.get(tc.dataset+"_train")
    print(tc.category, list(enumerate(MetadataCatalog.get(tc.dataset+"_train").thing_classes)))

In [ ]:
CustomConfig.pretty()

In [ ]:
# CustomConfig.save()

# File inspection

In [ ]:
!du -sh /content/gdrive/

In [ ]:
!ls /content/gdrive/MyDrive/

In [ ]:
!du -h --max-depth=1 /content/gdrive/MyDrive/ | sort -hr

In [ ]:
!du -h --max-depth=1 /content/gdrive/MyDrive/4YP\ Output/detectron | sort -hr

In [ ]:
!ls -lh /content/gdrive/MyDrive/4YP\ Output/detectron/vertical0

In [ ]:
!find . -type f -name "model_final.pth" -prune -exec du -sh {} \;

# Reload PDM

In [ ]:
%cd /content/detectron2_pdm/
!git diff
%cd /content/

In [ ]:
%cd /content/detectron2_pdm/
!git reset HEAD --hard
!git pull
%cd /content/

In [ ]:
detectron2_pdm = reload(detectron2_pdm)
import detectron2_pdm
pdm = reload(pdm)
from detectron2_pdm import PDM_Evaluator as pdm
PDM_Evaluator = pdm.PDM_Evaluator
ct = reload(ct)
from detectron2_pdm import CustomTrainer as ct
CustomTrainer = ct.CustomTrainer
cc = reload(cc)
from detectron2_pdm import CustomConfig as cc
CustomConfig = cc.CustomConfig
Datasets = reload(Datasets)
from detectron2_pdm import Datasets
Console = reload(Console)
from detectron2_pdm import Console
Main = reload(Main)
from detectron2_pdm import Main
Visualise = reload(Visualise)
from detectron2_pdm import Visualise
setupCC()

In [ ]:
# !rm -rf /detectron2_pdm/

# Dataset Inspection

In [ ]:
!ls /content/gdrive/MyDrive/Share/4YPDatasets/

In [ ]:
DatasetCatalog.list()

In [ ]:
!python /content/detectron2_pdm/cococount.py vertical_300_train.json vertical_300_val.json vertical_300_test.json ground_200_train.json ground_200_val.json ground_200_test.json

In [ ]:
!python /content/detectron2_pdm/cococount.py G/*.json

In [ ]:
Datasets.clear()

## Counts

In [ ]:
# DatasetCatalog.clear()
# # register_coco_instances("ground_200_train", {}, "/content/ground_200_train.json", "/content/mask_ground/")
# # register_coco_instances("ground_200_val", {}, "/content/ground_200_val.json", "/content/mask_ground/")
# # register_coco_instances("ground_200_test", {}, "/content/ground_200_test.json", "/content/mask_ground/")
# # register_coco_instances("vertical_300_train", {}, "/content/vertical_300_train.json", "/content/mask_vertical/")
# # register_coco_instances("vertical_300_val", {}, "/content/vertical_300_val.json", "/content/mask_vertical/")
# # register_coco_instances("vertical_300_test", {}, "/content/vertical_300_test.json", "/content/mask_vertical/")
for i in ["ground_200_train", "ground_200_val", "ground_200_test", "vertical_300_train", "vertical_300_val", "vertical_300_test"]:
    print(i)
    build.print_instances_class_histogram(DatasetCatalog.get(i), MetadataCatalog.get(i).thing_classes)

# Train

In [ ]:
!rm -rf /content/outputs/M_train/

In [ ]:
CustomConfig.modelWeights

In [ ]:
pprint(Main.get_cfg().MODEL.WEIGHTS)

In [ ]:
logging.getLogger("detectron2").setLevel(logging.DEBUG)
Main.train(iterations=21000, resume=True, save=True)

In [ ]:
# !cp -rf /content/outputs/vertical1 /content/gdrive/My\ Drive/4YP\ Output/detectron

In [ ]:
# os.system(f"cp -rf /content/outputs/{CustomConfig.category}{outputn} {CustomConfig.driveOutputs}")

In [ ]:
# CustomConfig.save()

# Tensorboard

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir /content/outputs/

# Evaluation

In [ ]:
log = logging.getLogger("detectron2")
ll = log.getEffectiveLevel()
log.setLevel(logging.WARNING)

cfg = Main.get_cfg(weights_file="model_final.pth", load=False)
results = Main.evaluate(cfg)
pprint(results)
beep()

log.setLevel(ll)

In [ ]:
print(results)
print(str(results).replace("nan", "np.nan"))

In [ ]:
# print(f"{CustomConfig.category}{outputn} = {Main.evaluate_all_checkpoints(outputn)}")

# Visualise

In [ ]:
for tc in CustomConfig.testingConfigs:
    DatasetCatalog.get(tc.dataset+"_test")
    print(tc.folder, MetadataCatalog.get(tc.dataset+"_test").thing_classes)

In [ ]:
Visualise.compare(filterAnnotation=0, subset="_val", original=True)

In [ ]:
Visualise.compare(random=3, original=False)

In [ ]:
Visualise.compare(filterAnnotation=0, subset="_test")

In [ ]:
Visualise.compare(save=True)
beep()

In [ ]:
f"/content/outputs/{CustomConfig.testingConfigs[0].folder}/{base}"

### Metadata

In [ ]:
MetadataCatalog.get("vertical_300_test").thing_classes

In [ ]:
MetadataCatalog.get("ground_200_train").thing_classes

In [ ]:
DatasetCatalog.get("ground_200_train")[0]["annotations"][1]